## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-05-06-59-58 +0000,nypost,Trump’s emotional call to parents of fallen St...,https://nypost.com/2026/02/05/us-news/trump-ca...
1,2026-02-05-06-56-09 +0000,bbc,Social supermarket aims to provide affordable ...,https://www.bbc.com/news/articles/cn7j334jpypo...
2,2026-02-05-06-51-00 +0000,wsj,ArcelorMittal Eyes Boost From Europe’s Steel P...,https://www.wsj.com/business/earnings/arcelorm...
3,2026-02-05-06-51-00 +0000,wsj,Spain’s BBVA Profit Rises on Loan Growth,https://www.wsj.com/business/earnings/spains-b...
4,2026-02-05-06-49-14 +0000,bbc,Workers urge Target and US firms to speak up o...,https://www.bbc.com/news/articles/c4g4y4gwjpeo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2400/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,38
166,epstein,32
124,new,26
145,immigration,14
13,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
214,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...,97
282,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...,90
85,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...,89
63,2026-02-05-02-40-34 +0000,nypost,Trump ‘deploying all resources’ to help find S...,https://nypost.com/2026/02/04/us-news/trump-de...,89
117,2026-02-04-23-11-15 +0000,nypost,Trump gets help in his critical minerals race ...,https://nypost.com/2026/02/04/us-news/house-pa...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
214,97,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...
282,90,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...
85,57,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...
107,49,2026-02-04-23-48-00 +0000,wsj,Federal Reserve governor Lisa Cook sees a grea...,https://www.wsj.com/economy/central-banking/fe...
229,46,2026-02-04-19-24-56 +0000,startribune,Federal lawyer tells judge ‘job sucks’ amid ov...,https://www.startribune.com/federal-lawyer-tel...
69,44,2026-02-05-02-06-01 +0000,nypost,Furious lawmakers slam Gavin Newsom for trying...,https://nypost.com/2026/02/04/us-news/californ...
313,41,2026-02-04-14-52-00 +0000,wsj,"America’s private sector added 22,000 jobs las...",https://www.wsj.com/economy/jobs/adp-numbers-s...
147,41,2026-02-04-22-17-00 +0000,wsj,Washington Post Lays Off One-Third of Staff in...,https://www.wsj.com/business/media/washington-...
117,41,2026-02-04-23-11-15 +0000,nypost,Trump gets help in his critical minerals race ...,https://nypost.com/2026/02/04/us-news/house-pa...
259,38,2026-02-04-17-43-52 +0000,nypost,Second person dies during Japan skiing trip — ...,https://nypost.com/2026/02/04/world-news/secon...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
